In [1]:
import cv2
import subprocess
import json
import os

In [3]:
def get_video_properties_cv2(video_path):
    """
    Gets basic video properties using OpenCV.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_seconds = frame_count / fps if fps > 0 else 0

    cap.release()
    return {
        "width": width,
        "height": height,
        "fps": fps,
        "frame_count": frame_count,
        "duration_seconds": duration_seconds
    }

def get_video_codec_info_ffmpeg(video_path):
    """
    Gets detailed codec information using FFprobe (part of FFmpeg).
    Requires FFmpeg to be installed and in the system's PATH.
    """
    try:
        # Command to get stream information in JSON format
        cmd = [
            'ffprobe',
            '-v', 'quiet',
            '-print_format', 'json',
            '-show_streams',
            '-show_format',
            video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        info = json.loads(result.stdout)

        video_stream = None
        for stream in info.get('streams', []):
            if stream.get('codec_type') == 'video':
                video_stream = stream
                break

        if video_stream:
            codec_name = video_stream.get('codec_name')
            profile = video_stream.get('profile')
            bit_rate = int(video_stream.get('bit_rate', 0)) if 'bit_rate' in video_stream else None
            avg_frame_rate = video_stream.get('avg_frame_rate')
            chroma_loc = video_stream.get('chroma_location')
            pix_fmt = video_stream.get('pix_fmt') # Pixel format, often indicates chroma subsampling (e.g., yuv420p)
            tags = video_stream.get('tags', {})
            encoder = tags.get('encoder')

            # Get overall format info for bit rate
            format_info = info.get('format', {})
            format_bit_rate = int(format_info.get('bit_rate', 0)) if 'bit_rate' in format_info else None
            format_size = int(format_info.get('size', 0)) if 'size' in format_info else None

            return {
                "codec_name": codec_name,
                "profile": profile,
                "bit_rate_stream": bit_rate, # Specific to the video stream
                "bit_rate_format": format_bit_rate, # Overall file bitrate
                "file_size": format_size,
                "avg_frame_rate_ffmpeg": avg_frame_rate,
                "chroma_location": chroma_loc,
                "pixel_format": pix_fmt,
                "encoder": encoder
            }
        else:
            return None

    except FileNotFoundError:
        print("Error: ffprobe not found. Please ensure FFmpeg is installed and in your system's PATH.")
        return None
    except subprocess.CalledProcessError as e:
        print(f"Error running ffprobe: {e}")
        print(f"Stderr: {e.stderr}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing ffprobe output: {e}")
        print(f"FFprobe output: {result.stdout}")
        return None

def characterize_video(video_path):
    """
    Combines OpenCV and FFprobe information to characterize a video.
    """
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return None

    print(f"\n--- Analyzing Video: {os.path.basename(video_path)} ---")

    cv2_props = get_video_properties_cv2(video_path)
    if cv2_props:
        print("\n--- Basic Properties (from OpenCV) ---")
        print(f"  Resolution: {cv2_props['width']}x{cv2_props['height']}")
        print(f"  Frame Rate (FPS): {cv2_props['fps']:.2f}")
        print(f"  Total Frames: {cv2_props['frame_count']}")
        print(f"  Duration: {cv2_props['duration_seconds']:.2f} seconds")

    ffmpeg_info = get_video_codec_info_ffmpeg(video_path)
    if ffmpeg_info:
        print("\n--- Detailed Codec Information (from FFprobe) ---")
        print(f"  Codec Name: {ffmpeg_info.get('codec_name', 'N/A').upper()}") # Should be HEVC or h265
        print(f"  Profile: {ffmpeg_info.get('profile', 'N/A')}")
        # Convert bitrates to Mbps for readability
        stream_bitrate_mbps = (ffmpeg_info.get('bit_rate_stream') / 1_000_000) if ffmpeg_info.get('bit_rate_stream') else 'N/A'
        format_bitrate_mbps = (ffmpeg_info.get('bit_rate_format') / 1_000_000) if ffmpeg_info.get('bit_rate_format') else 'N/A'
        file_size_mb = (ffmpeg_info.get('file_size') / (1024 * 1024)) if ffmpeg_info.get('file_size') else 'N/A'


        print(f"  Video Stream Bitrate: {stream_bitrate_mbps:.2f} Mbps" if isinstance(stream_bitrate_mbps, float) else f"  Video Stream Bitrate: {stream_bitrate_mbps}")
        print(f"  Overall File Bitrate: {format_bitrate_mbps:.2f} Mbps" if isinstance(format_bitrate_mbps, float) else f"  Overall File Bitrate: {format_bitrate_mbps}")
        print(f"  File Size: {file_size_mb:.2f} MB" if isinstance(file_size_mb, float) else f"  File Size: {file_size_mb}")

        print(f"  Average Frame Rate (FFmpeg): {ffmpeg_info.get('avg_frame_rate_ffmpeg', 'N/A')}")
        print(f"  Chroma Location: {ffmpeg_info.get('chroma_location', 'N/A')}")
        print(f"  Pixel Format: {ffmpeg_info.get('pixel_format', 'N/A')}")
        print(f"  Encoder: {ffmpeg_info.get('encoder', 'N/A')}")
    else:
        print("Could not retrieve detailed codec information (FFprobe might be missing or failed).")

    return {"cv2_props": cv2_props, "ffmpeg_info": ffmpeg_info}

In [ ]:
if __name__ == "__main__":
    # --- IMPORTANT: Configure your video file path here ---
    # Example: Path to a single HEVC video file
    # video_file_path = "path/to/your/hevc_video.mp4" # Or .mkv, .mov, etc.

    # --- OR ---
    # Example: To process multiple videos in a directory
    # video_directory = "path/to/your/video_dataset_folder"
    # video_extensions = ['.mp4', '.mkv', '.mov', '.hevc'] # Add other extensions as needed

    # If you want to process a single file:
    # characterize_video(video_file_path)

    # If you want to process all files in a directory:
    # for root, _, files in os.walk(video_directory):
    #     for file in files:
    #         if any(file.lower().endswith(ext) for ext in video_extensions):
    #             full_path = os.path.join(root, file)
    #             characterize_video(full_path)

    # For testing, let's use a placeholder. You MUST change this.
    my_video_to_analyze = "vid.avi" # <--- !!! CHANGE THIS !!!

    if os.path.exists(my_video_to_analyze):
        characterize_video(my_video_to_analyze)
    else:
        print(f"\nWARNING: Video file '{my_video_to_analyze}' not found.")
        print("Please update 'my_video_to_analyze' in the script with the correct path to your HEVC video.")
        print("Example: my_video_to_analyze = 'C:/Users/YourUser/Videos/my_rppg_dataset/hevc_clip_01.mp4'")
        print("You can also uncomment the directory processing loop to analyze all videos in a folder.")


--- Analyzing Video: vid.avi ---

--- Basic Properties (from OpenCV) ---
  Resolution: 640x480
  Frame Rate (FPS): 29.26
  Total Frames: 1547
  Duration: 52.86 seconds

--- Detailed Codec Information (from FFprobe) ---
  Codec Name: RAWVIDEO
  Profile: None
  Video Stream Bitrate: 215.90 Mbps
  Overall File Bitrate: 215.78 Mbps
  File Size: 1359.78 MB
  Average Frame Rate (FFmpeg): 14632053/500000
  Chroma Location: None
  Pixel Format: bgr24
  Encoder: None
